### LSTM

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('BTC-Indicators.csv', parse_dates=['Date'], index_col=['Date'])
data

,Close,sma,ema20,ema50,ema200,rsi,macd,macd_signal,Close_future,Price_Up
Date,,,,,,,,,,
2015-04-04,253.697006,266.058734,255.661524,258.734938,307.338692,47.508324,-4.193133,-3.906538,260.597992,1
2015-04-05,260.597992,265.654567,256.131663,258.807999,306.873610,52.432292,-3.252122,-3.775655,255.492004,0
2015-04-06,255.492004,264.962268,256.070744,258.677960,306.362350,48.785829,-2.885117,-3.597547,253.179993,0
2015-04-07,253.179993,264.256468,255.795434,258.462354,305.833173,47.185618,-2.749132,-3.427864,245.022003,0
2015-04-08,245.022003,262.770301,254.769393,257.935281,305.228086,41.956153,-3.262042,-3.394700,243.675995,0
...,...,...,...,...,...,...,...,...,...,...
2024-11-30,96449.054688,86570.707812,91648.316009,82261.362670,68184.502386,65.500818,5191.352498,5794.364534,97279.789062,1
2024-12-01,97279.789062,87497.285156,92184.646776,82850.320568,68474.007726,66.656792,5022.591458,5640.009918,95865.304688,0
2024-12-02,95865.304688,88383.152865,92535.185625,83360.712102,68746.557944,62.798383,4720.297400,5456.067415,96002.164062,1


In [2]:
features = ['Close', 'sma', 'ema20', 'ema50', 'ema200', 'rsi', 'macd', 'macd_signal']
target = ['Price_Up']
# Separar X e y
X = data[features]
y = data[target].to_numpy()
y.shape, y

((3533, 1),
 array([[1],
        [0],
        [0],
        ...,
        [1],
        [1],
        [1]], dtype=int64))

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X) # Ya lo convierte en un array de numpy 
X_scaled.shape, X_scaled

((3533, 8),
 array([[-0.93202605, -0.94080889, -0.93947655, ..., -0.45317299,
         -0.153607  , -0.1574351 ],
        [-0.93170146, -0.94082848, -0.93945397, ..., -0.10429159,
         -0.15279471, -0.15731524],
        [-0.93194162, -0.94086202, -0.9394569 , ..., -0.362657  ,
         -0.15247791, -0.15715212],
        ...,
        [ 3.56511431,  3.32887102,  3.49210365, ...,  0.6301845 ,
          3.92460032,  4.84291506],
        [ 3.57155156,  3.37290181,  3.50796043, ...,  0.64598744,
          3.68314866,  4.65691486],
        [ 3.70166904,  3.42290216,  3.53495947, ...,  0.948451  ,
          3.64078593,  4.49912575]]))

In [4]:
sequence_length = 30 # Ventana temporal de 30 dias 
X_sequences = []
y_sequences = []
for i in range(sequence_length, len(X_scaled)): # X_scaled tiene dimension de 1000, X_sequences tendra 1000 - 30(sequence_leght)
    X_sequences.append(X_scaled[i - sequence_length: i])
    y_sequences.append(y[i])
X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)
X_sequences.shape, y_sequences.shape


((3503, 30, 8), (3503, 1))

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_sequences, y_sequences, test_size=0.01, shuffle=False)

In [6]:
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization, Bidirectional  # type: ignore
from tensorflow.keras.optimizers import Adam  # type: ignore

model = Sequential()
model.add(Bidirectional(LSTM(units=100, return_sequences=False), input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(BatchNormalization())

# model.add(Bidirectional(LSTM(units=100, return_sequences=False)))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())

model.add(Dense(units=15, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


c:\Users\ivoto\miniconda3\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
history = model.fit(
    X_train, y_train,
    epochs=25,
    batch_size=32,
    validation_data=(X_test, y_test)
)

Epoch 1/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.4942 - loss: 0.7658 - val_accuracy: 0.5833 - val_loss: 0.6865
Epoch 2/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4890 - loss: 0.7263 - val_accuracy: 0.5278 - val_loss: 0.6923
Epoch 3/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5083 - loss: 0.7097 - val_accuracy: 0.4167 - val_loss: 0.7437
Epoch 4/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5289 - loss: 0.7001 - val_accuracy: 0.5278 - val_loss: 0.6845
Epoch 5/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5211 - loss: 0.6983 - val_accuracy: 0.5833 - val_loss: 0.6875
Epoch 6/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5263 - loss: 0.6971 - val_accuracy: 0.5278 - val_loss: 0.6929
Epoch 7/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5241 - loss: 0.6955 - val_accuracy: 0.4167 - val_loss: 0.7219
Epoch 8/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5332 - loss: 0.6920 - val_accu

In [8]:
# 9. Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Precisión del modelo LSTM: {accuracy:.2f}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4236 - loss: 0.6967
Precisión del modelo LSTM: 0.42


In [9]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
              precision    recall  f1-score   support

           0       0.42      1.00      0.59        15
           1       0.00      0.00      0.00        21

    accuracy                           0.42        36
   macro avg       0.21      0.50      0.29        36
weighted avg       0.17      0.42      0.25        36

[[15  0]
 [21  0]]


c:\Users\ivoto\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ivoto\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ivoto\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Implementacion con Pytorch

In [10]:
# Convertimos los datos de Numpy a Tensores de Pytorch 
import torch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

X_train_tensor.shape, X_test_tensor.shape, y_train_tensor.shape, y_test_tensor.shape 

(torch.Size([3467, 30, 8]),
 torch.Size([36, 30, 8]),
 torch.Size([3467, 1]),
 torch.Size([36, 1]))

In [11]:
# Creamos Datasets y Dataloaders de Pytorch
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor) # Instanciamos la clase TendorDataSet 
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size, shuffle=False) # Instanciamos la clase DataLoader
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

for batch_X, batch_y in train_loader:
    print("Batch X shape:", batch_X.shape)
    print("Batch y shape:", batch_y.shape)
      # Solo el primer batch
    break


Batch X shape: torch.Size([32, 30, 8])
Batch y shape: torch.Size([32, 1])


In [17]:
import torch
import torch.nn as nn 

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print (device)

class MiLSTM (nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size ):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        # Definimos 1 capa LSTM y un Full Connected 
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True) #Ej [8, 64, 2, (batch_size, seq_length, hidden_size)]
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Estados ocultos y celdas en 0 (num_layers, batch_size, hidden_size)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) # Ej [2, 32, 64]
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) # Ej [2, 32, 64]

        # Pasa la entrada por la capa LSTM 
        out, _ = self.lstm(x, (h0,c0)) #LLamo a la instancia de lstm (llamo a su metodo __call__ (no necesita nombre)) (batch_size, seq_length, hidden_size) [32, 30, 64]

        #Seleccionamos la ultima salida de la secuencia
        out = out[:,-1,:] # (batch_size , hidden_size) [32,64]

        #Pasa por la fc
        out = self.fc(out) # (batch_size, output_size) Ej [32,1]

        return out

# Parametros del modelo 
input_size = 8 
hidden_size = 64 
num_layers  = 2
output_size = 1 # Clasificacion 

model = MiLSTM(input_size, hidden_size, num_layers, output_size).to(device)
print(model) 

cuda
MiLSTM(
  (lstm): LSTM(8, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)


In [20]:
# Definir la función de pérdida y el optimizador
creterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
num_epochs = 50 
model.train()

for epoch in range(num_epochs):
    for batch_X, batch_y in train_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        outputs = model(batch_X).squeeze()

        loss = creterion(outputs, batch_y.squeeze())

        loss.backward()

        optimizer.step()

        optimizer.zero_grad()
    if (epoch + 1 ) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')





Epoch [10/50], Loss: 0.6501
Epoch [20/50], Loss: 0.5722
Epoch [30/50], Loss: 0.5788
Epoch [40/50], Loss: 0.5572
Epoch [50/50], Loss: 0.8525


In [21]:
model.eval()  # Modo de evaluación
with torch.no_grad():
    correct = 0
    total = 0
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        outputs = model(batch_X).squeeze()  # Logits
        probs = torch.sigmoid(outputs)       # Probabilidades
        predicted = torch.round(probs)       # Predicciones binarias (0 o 1)

        total += batch_y.size(0)
        correct += (predicted == batch_y.squeeze()).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on test set: {accuracy:.2f}%')

Accuracy on test set: 58.33%
